This model is meant to forecast the outcome of the standings here: https://udisclive.com/players?t=standings&z=dgpt 
Logic for how to attribute points is here: https://udisc.com/blog/post/how-disc-golf-pro-tour-points-work-why-they-matter?fbclid=IwAR1VwYCkl7DCkgc93G5qujSDxCSWqg5HMWLv7dVhu_c4GXchW_P7fJO7MSo

To do so, I need to:
- load in events, and details about those events
- load in the players playing in those events, and details about their skill levels
- generate N runs of a model that forecasts each player's results N times at each event they are entered in
- use the generated results to assign points to the players
- sum the points based on the DGPT rules
- aggregate the ranks for each player at the end of the year (for example, see if Paige Pierce has a 90% chance of 1st place, 5% chance of 2nd place, etc.)

Things that inspired this / what the output should look like
- 538 soccer: https://projects.fivethirtyeight.com/soccer-predictions/champions-league/
- my own attempt at this same thing but in google sheets: https://docs.google.com/spreadsheets/d/19IwKCt5DI77koId916DawdJd1mNM7RQi3Nzlpmsa23Q/edit#gid=0 (see  ForecastFinal tab. This doc does ALL of the logic i want in here, but it's limited to N=50 runs due to how long it takes the formulas to update)

Load In Packages

In [1]:
import pandas as pd
import numpy as np

Load in Events - Details from PDGA.com using Event ID as unique Identifier
Would eventually like to make this step scrape the site, but we can get there later

Load Players signed up for events - Details from PDGA.com with PDGA# and Event ID as unique Identifier
This also needs to be scraped eventually, and will need to include a "load date" column since ratings change once a month. For now, just a csv.

Load Player Scores
Placeholder - eventually I'll need to include the rounds that have already occured (since those scores are 100% likely to happen). I haven't written any code to account for this yet.

In [2]:
events = pd.read_csv('events.csv')

In [3]:
eventplayers = pd.read_csv('EventPlayersMPO20210324.csv')

In [4]:
pointsLogic = pd.read_csv('PointsLogic.csv')

In [5]:
###eventplayerscores = pd.read_csv('eventplayerscores.csv')

Event Mean Regression Creation

This cell creates a simple mean regression - the farther out the event, the more likely player ratings are to change. This variable is mostly a guess right now, and is not coming from any valid regression of how player ratings change over time. Also I had to take the absolute value so that the event that already occured didn't break everything. In real life, I should just have those scores already loaded in.

In [6]:
events['Event Date'] = pd.to_datetime(events['Event Date'])
events['Today'] = pd.to_datetime("now")
events['Time to Event'] = events['Event Date'] - events['Today']
events['Time to Event Number'] = pd.to_numeric(events['Time to Event'])/(1000000000*60*60*24)
events['Event Mean Regression'] = np.log10(np.absolute(events['Time to Event Number']))

Generate Average Rating At Each Event - useful on its own, but needed to calculate the avg round scores. Ideally would use historical data here, but really just need a baseline so I have something to work with

Declare StdDev - 6.82 is a number I grabbed from a couple 2020 tournaments to use later on in generating the normal distribution and (you guessed it) probably needs more work to be better.

In [7]:
eventplayers['Rating'] = pd.to_numeric(eventplayers['Rating'])
means = eventplayers.groupby('Event ID')['Rating'].mean()
events = pd.merge(events,means,on = 'Event ID',how = 'left')
events['StdDev'] = 6.82

###events

Calculate Single Round Expected Scores

Here we declare "rating points per stroke" so that we can compare ex. a 1000 to 900 rated player in a single round. Then we turn that into the basic "expected round score" that can be used in the upcoming randomization.

In [8]:
rating_points_per_stroke = 6
eventplayerratings = pd.merge(events,eventplayers, on = 'Event ID',how = 'right')
eventplayerratings['Single Round Expected Score'] = -1*((eventplayerratings['Rating_y']-eventplayerratings['Rating_x'])/(rating_points_per_stroke+eventplayerratings['Event Mean Regression']))
eventplayerratings = eventplayerratings.drop(['FinalRank'], axis = 1)

eventplayerratings

,Event Name,Event ID,EventType,EventLength,Event Date,Today,Time to Event,Time to Event Number,Event Mean Regression,Rating_x,StdDev,Name,PDGANumber,Rating_y,Single Round Expected Score
0,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,Geoff Hungerford,688,953,6.798189
1,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,Jon Drummond,7350,974,4.199850
2,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,Brad Schick,7992,1013,-0.625637
3,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,Nikko Locastro,11534,1030,-2.729054
4,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,David Feldberg,12626,1023,-1.862941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623,Portland,48686,Elite,3,2021-06-04,2021-03-24 04:40:21.330487,71 days 19:19:38.669513,71.805309,1.856157,1002.333333,6.82,Terran Ferguson,111010,996,0.806162
1624,Portland,48686,Elite,3,2021-06-04,2021-03-24 04:40:21.330487,71 days 19:19:38.669513,71.805309,1.856157,1002.333333,6.82,Kenny Shelton,112835,985,2.206338
1625,Portland,48686,Elite,3,2021-06-04,2021-03-24 04:40:21.330487,71 days 19:19:38.669513,71.805309,1.856157,1002.333333,6.82,Joshua Hatcher,115038,995,0.933451
1626,Portland,48686,Elite,3,2021-06-04,2021-03-24 04:40:21.330487,71 days 19:19:38.669513,71.805309,1.856157,1002.333333,6.82,Kirby Snyder,115165,975,3.479225


Generate Random Scores

In [9]:
###Create N number of copies of the dataframe
eventplayereventratings = pd.concat([eventplayerratings for i in range(1000)],
          ignore_index=True)

eventplayerroundratings = pd.concat([eventplayerratings for i in range(1000)],
          ignore_index=True)

###Number each iteration of the model
eventplayereventratings['ModelRunNumber']= eventplayereventratings.groupby(['Event ID','PDGANumber'])['PDGANumber'].rank(method='first')
eventplayerroundratings['ModelRunNumber']= eventplayerroundratings.groupby(['Event ID','PDGANumber'])['PDGANumber'].rank(method='first')


###Copy the copies by # of rounds, then # the rounds
eventplayerroundratings = eventplayerroundratings.iloc[np.arange(len(eventplayerroundratings)).repeat(eventplayerroundratings['EventLength'])]
eventplayerroundratings['RoundNumber']= eventplayerroundratings.groupby(['Event ID','PDGANumber','ModelRunNumber'])['PDGANumber'].rank(method='first')

#Generate the Scores, and Round them to Integers
eventplayerroundratings['RandomScores'] = np.random.normal(
        eventplayerroundratings['Single Round Expected Score'].values,
        eventplayerroundratings['StdDev'].values)
eventplayerroundratings['RandomScores'] = eventplayerroundratings['RandomScores'].round(0)
eventplayerroundratings['RandomScores'] = eventplayerroundratings.RandomScores.astype(int)
eventplayerroundratings['RoundNumber'] = eventplayerroundratings.RoundNumber.astype(int)
eventplayerroundratings['ModelRunNumber'] = eventplayerroundratings.ModelRunNumber.astype(int)

###eventplayerroundratings

In [10]:
ModelRunScore = eventplayerroundratings.groupby(['Event ID','PDGANumber','ModelRunNumber'],as_index=False).sum()
ModelRunScore = ModelRunScore.drop(['EventLength','Time to Event Number','Event Mean Regression','Rating_x','StdDev','Rating_y','Single Round Expected Score','RoundNumber'], axis = 1)
###ModelRunScore

In [11]:
eventplayereventratings = pd.merge(eventplayereventratings,ModelRunScore, on = ['Event ID','PDGANumber','ModelRunNumber'],how = 'left')
eventplayereventratings['ModelRunNumber'] = eventplayereventratings.ModelRunNumber.astype(int)

eventplayereventratings

,Event Name,Event ID,EventType,EventLength,Event Date,Today,Time to Event,Time to Event Number,Event Mean Regression,Rating_x,StdDev,Name,PDGANumber,Rating_y,Single Round Expected Score,ModelRunNumber,RandomScores
0,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,Geoff Hungerford,688,953,6.798189,1,23
1,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,Jon Drummond,7350,974,4.199850,1,2
2,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,Brad Schick,7992,1013,-0.625637,1,-11
3,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,Nikko Locastro,11534,1030,-2.729054,1,-4
4,DGLO,48338,Elite,3,2021-07-23,2021-03-24 04:40:21.330487,120 days 19:19:38.669513,120.805309,2.082086,1007.943548,6.82,David Feldberg,12626,1023,-1.862941,1,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627995,Portland,48686,Elite,3,2021-06-04,2021-03-24 04:40:21.330487,71 days 19:19:38.669513,71.805309,1.856157,1002.333333,6.82,Terran Ferguson,111010,996,0.806162,1000,-12
1627996,Portland,48686,Elite,3,2021-06-04,2021-03-24 04:40:21.330487,71 days 19:19:38.669513,71.805309,1.856157,1002.333333,6.82,Kenny Shelton,112835,985,2.206338,1000,1
1627997,Portland,48686,Elite,3,2021-06-04,2021-03-24 04:40:21.330487,71 days 19:19:38.669513,71.805309,1.856157,1002.333333,6.82,Joshua Hatcher,115038,995,0.933451,1000,15
1627998,Portland,48686,Elite,3,2021-06-04,2021-03-24 04:40:21.330487,71 days 19:19:38.669513,71.805309,1.856157,1002.333333,6.82,Kirby Snyder,115165,975,3.479225,1000,7


In [12]:
eventplayereventratings['EventRankBeforePlayoff'] = eventplayereventratings.groupby(['Event ID','ModelRunNumber'])['RandomScores'].rank(method='min')
eventplayereventratings['EventRankBeforePlayoff'] = eventplayereventratings.EventRankBeforePlayoff.astype(int)

###Ridiculous amount of code to break ties in playoffs. Two things it should do that it doesn't : account for strength of player, account for the low % chance that a playoff can't occur
number_of_ties_before_playoff = eventplayereventratings.groupby(['EventRankBeforePlayoff','Event ID','ModelRunNumber'],as_index=False)['EventRankBeforePlayoff'].size()
first_place_ties = pd.merge(eventplayereventratings,number_of_ties_before_playoff, on = ['Event ID','EventRankBeforePlayoff','ModelRunNumber'],how = 'right')
first_place_ties = first_place_ties[first_place_ties['size']>1]
first_place_ties = first_place_ties[first_place_ties['EventRankBeforePlayoff']==1]
first_place_ties = first_place_ties.drop(['Event Date','EventLength','Time to Event Number','Event Mean Regression','Rating_x','StdDev','Rating_y','Single Round Expected Score','RandomScores','Event Name','EventType','Today','Time to Event','Name'], axis = 1)
first_place_ties['Random'] = np.random.rand(len(first_place_ties.index))
first_place_ties['RankAfterPlayoff']= first_place_ties.groupby(['Event ID','ModelRunNumber'])['Random'].rank(method='first')
first_place_ties['RankAfterPlayoff'] = first_place_ties.RankAfterPlayoff.astype(int)
first_place_ties['RankAfterPlayoffWithSecondPlaceTies']= first_place_ties['RankAfterPlayoff']
first_place_ties['RankAfterPlayoffWithSecondPlaceTies'] = np.where((first_place_ties.RankAfterPlayoff >= 2),2,first_place_ties.RankAfterPlayoffWithSecondPlaceTies)
first_place_ties = first_place_ties.drop(['EventRankBeforePlayoff','Random','RankAfterPlayoff','size'], axis = 1)

first_place_ties

,Event ID,PDGANumber,ModelRunNumber,RankAfterPlayoffWithSecondPlaceTies
15,47446,45971,16,1
16,47446,57365,16,2
32,47446,41760,32,2
33,47446,85850,32,1
42,47446,38008,41,2
...,...,...,...,...
17344,49214,50312,966,1
17350,49214,41760,972,2
17351,49214,45971,972,1
17378,49214,41760,999,1


In [13]:
eventplayereventratings = pd.merge(eventplayereventratings,first_place_ties, on = ['Event ID','PDGANumber','ModelRunNumber'],how = 'outer')
eventplayereventratings.RankAfterPlayoffWithSecondPlaceTies.fillna(eventplayereventratings.EventRankBeforePlayoff, inplace = True)
eventplayereventratings['RankAfterPlayoffWithSecondPlaceTies'] = eventplayereventratings.RankAfterPlayoffWithSecondPlaceTies.astype(int)

In [14]:


### calculate points for each run
eventplayereventratings = pd.merge(eventplayereventratings,eventplayers, on = ['Event ID','PDGANumber','Name'],how = 'left')
eventplayereventratings = eventplayereventratings.drop(['Rating_x','EventRankBeforePlayoff','StdDev','Today','Time to Event','Rating_y'], axis = 1)
eventplayereventratings.FinalRank = np.where(eventplayereventratings.FinalRank.isnull(),eventplayereventratings.RankAfterPlayoffWithSecondPlaceTies,eventplayereventratings.FinalRank)
eventplayereventratings['EventRankForPoints'] = eventplayereventratings.groupby(['Event ID','ModelRunNumber'])['FinalRank'].rank(method='first')
eventplayereventratings['EventRankForPoints'] = eventplayereventratings.EventRankForPoints.astype(int)
eventplayereventratings = pd.merge(eventplayereventratings,pointsLogic, on = 'EventRankForPoints',how = 'left')
eventplayereventratings = eventplayereventratings.drop(['EventRankForPoints'], axis = 1)

eventplayereventratings = eventplayereventratings.drop(['RankAfterPlayoffWithSecondPlaceTies'], axis = 1)
eventplayereventratings['FinalRank'] = eventplayereventratings.FinalRank.astype(int)
###find the number of ties
number_of_ties = eventplayereventratings.groupby(['FinalRank','Event ID','ModelRunNumber'],as_index=False)['FinalRank'].size()
number_of_ties.rename(columns = {'size':'NumberOfTies'}, inplace = True)
###merge the number of ties back into the thing
eventplayereventratings = pd.merge(eventplayereventratings,number_of_ties, on = ['Event ID','FinalRank','ModelRunNumber'],how = 'right')

eventplayereventratings

,Event Name,Event ID,EventType,EventLength,Event Date,Time to Event Number,Event Mean Regression,Name,PDGANumber,Single Round Expected Score,ModelRunNumber,RandomScores,FinalRank,Rating,Points,NumberOfTies
0,PCS Sula Open,47446,Elite,4,2021-07-07,104.805309,2.020383,Garrett Gurthie,13864,-5.400456,1,-58,1,1036,100,1
1,PCS Sula Open,47446,Elite,4,2021-07-07,104.805309,2.020383,Karl Johan Nybo,28903,-4.028950,2,-47,1,1025,100,1
2,PCS Sula Open,47446,Elite,4,2021-07-07,104.805309,2.020383,Richard Wysocki,38008,-7.520055,3,-53,1,1053,100,1
3,PCS Sula Open,47446,Elite,4,2021-07-07,104.805309,2.020383,Adam Hammes,57365,-5.774503,4,-68,1,1039,100,1
4,PCS Sula Open,47446,Elite,4,2021-07-07,104.805309,2.020383,Eagle Wynne McMahon,37817,-7.894102,5,-56,1,1056,100,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627995,Ledgestone,47981,Elite,4,2021-08-05,133.805309,2.126473,Kennan Johnson,77936,4.895016,996,43,154,970,1,1
1627996,Ledgestone,47981,Elite,4,2021-08-05,133.805309,2.126473,Matt Dollar,26045,1.203378,997,40,154,1000,1,1
1627997,Ledgestone,47981,Elite,4,2021-08-05,133.805309,2.126473,William Schram,127314,4.771962,998,47,154,971,1,1
1627998,Ledgestone,47981,Elite,4,2021-08-05,133.805309,2.126473,Sam Drummond,100689,1.572542,999,46,154,997,1,1


In [15]:
#Calculate Points for Ties
tiePoints = eventplayereventratings.groupby(['FinalRank','Event ID','EventType','ModelRunNumber','NumberOfTies'],as_index=False).sum()
tiePoints = tiePoints.drop(['EventLength','Time to Event Number','Event Mean Regression','PDGANumber','Single Round Expected Score','RandomScores'], axis = 1)
tiePoints['Actual Points'] = tiePoints['Points']/tiePoints['NumberOfTies']
tiePoints['Actual Points'] = np.where(tiePoints['EventType'] == 'Silver', tiePoints['Actual Points']/4,tiePoints['Actual Points'])
tiePoints = tiePoints.drop(['Points','NumberOfTies','EventType'], axis = 1)
tiePoints.query('ModelRunNumber == "1" and `Event ID` == "47877"')

,FinalRank,Event ID,ModelRunNumber,Rating,Actual Points
2000,1,47877,1,1050,100.000000
18000,2,47877,1,2067,80.000000
46778,4,47877,1,1035,69.000000
60240,5,47877,1,1054,64.000000
72204,6,47877,1,1035,60.000000
83669,7,47877,1,2047,55.500000
103514,9,47877,1,1053,52.000000
112826,10,47877,1,1021,50.000000
121914,11,47877,1,2055,47.000000
139548,13,47877,1,1019,44.000000


In [16]:
eventplayereventratings = pd.merge(eventplayereventratings,tiePoints, on = ['FinalRank','Event ID','ModelRunNumber'],how = 'left')
eventplayereventratings['EliteSeriesRank'] = eventplayereventratings[eventplayereventratings.EventType == 'Elite'].groupby(['PDGANumber','ModelRunNumber'])['Actual Points'].rank(method='first')
eventplayereventratings['EliteSeriesRank'] = eventplayereventratings['EliteSeriesRank'].fillna(100)
eventplayereventratings['EliteSeriesRank'] = eventplayereventratings.EliteSeriesRank.astype(int)
###eventplayereventratings[eventplayereventratings.EventType == 'Elite']
eventplayereventratings['SilverSeriesRank'] = eventplayereventratings[eventplayereventratings.EventType == 'Silver'].groupby(['PDGANumber','ModelRunNumber'])['Actual Points'].rank(method='first')
eventplayereventratings['SilverSeriesRank'] = eventplayereventratings['SilverSeriesRank'].fillna(100)
eventplayereventratings['SilverSeriesRank'] = eventplayereventratings.SilverSeriesRank.astype(int)


In [17]:
eventplayereventratings['KeepScore'] = np.where((eventplayereventratings['EliteSeriesRank']<=8)|(eventplayereventratings['SilverSeriesRank']<=3),'Yes','No')
eventplayereventratings.query('`Event ID` == "48686" and PDGANumber == "99455"')

,Event Name,Event ID,EventType,EventLength,Event Date,Time to Event Number,Event Mean Regression,Name,PDGANumber,Single Round Expected Score,...,RandomScores,FinalRank,Rating_x,Points,NumberOfTies,Rating_y,Actual Points,EliteSeriesRank,SilverSeriesRank,KeepScore


In [18]:
ModelRunScore = eventplayereventratings[eventplayereventratings.KeepScore =='Yes'].groupby(['PDGANumber','ModelRunNumber'],as_index=False).sum()
ModelRunScore = ModelRunScore.drop(['Event ID','EventLength','Time to Event Number','Event Mean Regression','Single Round Expected Score','RandomScores','NumberOfTies','Points','EliteSeriesRank','SilverSeriesRank'], axis = 1)

In [19]:
ModelRunScore

,PDGANumber,ModelRunNumber,FinalRank,Rating_x,Rating_y,Actual Points
0,688,1,112,953,1964,1.00
1,688,2,124,953,953,1.00
2,688,3,122,953,1952,1.00
3,688,4,60,953,4999,1.00
4,688,5,107,953,1961,1.00
...,...,...,...,...,...,...
609995,153363,996,87,959,1967,0.25
609996,153363,997,87,959,1967,0.25
609997,153363,998,87,959,1967,0.25
609998,153363,999,87,959,1967,0.25


In [20]:
ModelRunScore['FinalStandings'] = ModelRunScore.groupby(['ModelRunNumber'])['Actual Points'].rank(ascending=False,method='min').astype(int)
EventPlayersUnique = eventplayers.drop_duplicates(subset=['PDGANumber','Name'])
EventPlayersUnique = EventPlayersUnique.drop('FinalRank', axis = 1)
EventPlayersUnique

,Event ID,Name,PDGANumber,Rating
0,48338,Geoff Hungerford,688,953
1,48172,Jerry Goff,2431,970
2,48121,George Smith,4034,977
3,47446,Egil Sætren,4322,972
4,48172,Steven Rico,4666,1031
...,...,...,...,...
1623,47888,Randy Dueck,139619,949
1624,48283,Diego Lopez,140901,937
1625,48283,Thomas Grigg,148486,964
1626,48686,Shawn Persing,153081,989


In [21]:
ModelRunScore = pd.merge(ModelRunScore,EventPlayersUnique,on=['PDGANumber'],how = 'left')
ModelRunScore

,PDGANumber,ModelRunNumber,FinalRank,Rating_x,Rating_y,Actual Points,FinalStandings,Event ID,Name,Rating
0,688,1,112,953,1964,1.00,312,48338,Geoff Hungerford,953
1,688,2,124,953,953,1.00,317,48338,Geoff Hungerford,953
2,688,3,122,953,1952,1.00,307,48338,Geoff Hungerford,953
3,688,4,60,953,4999,1.00,314,48338,Geoff Hungerford,953
4,688,5,107,953,1961,1.00,308,48338,Geoff Hungerford,953
...,...,...,...,...,...,...,...,...,...,...
609995,153363,996,87,959,1967,0.25,481,47888,Jaden Rye,959
609996,153363,997,87,959,1967,0.25,476,47888,Jaden Rye,959
609997,153363,998,87,959,1967,0.25,476,47888,Jaden Rye,959
609998,153363,999,87,959,1967,0.25,476,47888,Jaden Rye,959


In [22]:
FinalResultsAggregated = pd.pivot_table(ModelRunScore,index=['Name','PDGANumber'],columns=['FinalStandings'],values=['Actual Points'],aggfunc='count',fill_value =0)
FinalResultsAggregated

Actual Points                                  \
FinalStandings                          1   2   3   4   5   6   7   8   9     
Name               PDGANumber                                                 
A.J. Herr          56485                  0   0   0   0   0   0   0   0   0   
A.J. Risley        25890                  0   0   0   0   0   0   0   0   0   
AJ Carey           61770                  0   0   0   0   0   0   0   0   1   
Aaron DeVries      62322                  0   0   0   0   0   0   0   0   0   
Aaron Gossage      35449                  0   0   0   1   0   2   1   5   3   
...                                     ...  ..  ..  ..  ..  ..  ..  ..  ..   
Zack Finwall       89111                  0   0   0   0   0   0   0   0   0   
Zack Mullin        100986                 0   0   0   0   0   0   0   0   0   
Zackeriath Johnson 50312                  0   0   0   0   0   0   0   1   2   
Zane Letts         57544                  0   0   0   0   0   0   0   0   0   
Øyvind Jarnes      41884                  0   0   0   0   0   0   0   0   0   

                                   ...                                          
FinalStandings                10   ... 480 481 482 483 484 485 486 487 488 489  
Name               PDGANumber      ...                                          
A.J. Herr          56485        0  ...   0   0   0   0   0   0   0   0   0   0  
A.J. Risley        25890        0  ...   0   0   0   0   0   0   0   0   0   0  
AJ Carey           61770        0  ...   0   0   0   0   0   0   0   0   0   0  
Aaron DeVries      62322        0  ...   0   0   0   0   0   0   0   0   0   0  
Aaron Gossage      35449        7  ...   0   0   0   0   0   0   0   0   0   0  
...                            ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  
Zack Finwall       89111        0  ...   0   0   0   0   0   0   0   0   0   0  
Zack Mullin        100986       0  ...  95  67  49  38  36  11   5   6   4   1  
Zackeriath Johnson 50312        0  ...   0   0   0   0   0   0   0   0   0   0  
Zane Letts         57544        0  ...   0   0   0   0   0   0   0   0   0   0  
Øyvind Jarnes      41884        0  ...   0   0   0   0   0   0   0   0   0   0  

[610 rows x 356 columns]

In [24]:
FinalResultsAggregated.to_csv('FinalResultsAggregated2.csv')